In [1]:
import sail.core
import json
import getpass

password = "SailPassword@123"

sail.core.login("lbart@igr.com", password, 6200, "52.147.201.1")


201

In [2]:
#########################################################
# Allocate our job and load our SafeObjects
#########################################################

sail.core.load_safe_objects("../SafeObjects")
dg_test_object_guid = "479F9891BBDE41C2B8A06CF2E29F32AC"
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
dg_sum_column_guid = "442D0290DEA14CE4B70FB31B891F20B6"

job_id = sail.core.run_job(dg_test_object_guid)
print("Job ID: {} ".format(job_id))

Job ID: {4AF0483A-580C-4015-9995-BF48AC51820D} 


In [6]:
#########################################################
# Extract our identifiers
#########################################################
json_datasets = json.loads(sail.core.get_datasets())
json_digital_contracts = json.loads(sail.core.get_digital_contracts())

provision_dc = ""
provision_ds = ""
my_table_guid = ""
for dataset in json_datasets:
    if(json_datasets[dataset]["DatasetName"] == "SanityDataset"):
        provision_ds = dataset
        my_table_guid = list(json_datasets[dataset]["Tables"].keys())[0]

for digital_contract in json_digital_contracts:
    trimmed_ds = provision_ds.replace('{','').replace('}','')
    if (json_digital_contracts[digital_contract]["DatasetGuid"] == trimmed_ds):
        provision_dc = digital_contract
    
print("DC {} DS {} TABLE {}".format(provision_dc, provision_ds, my_table_guid))

DC {162BB07D-4BAC-432E-929D-9084AAE26CC2} DS {2071FE30-A497-0F47-B2DC-F0A8BDB7AED3} TABLE 2547D6F1-AEDB-E741-AA48-7630ABE72F55


In [7]:
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))
#job_status = sail.core.get_job_status(second_job_id)
#print("{}:{}".format(second_job_id,job_status))

{4AF0483A-580C-4015-9995-BF48AC51820D}:Missing Input Parameters


In [8]:
first_provision = sail.core.provision_secure_computational_node(provision_dc, provision_ds, "Standard_D4s_v4")

print("Provision ID: {}".format(first_provision))


Provision ID: {
    "Status": true,
    "SCNGuid": "{43A800D3-CF11-4BA0-99ED-A0EA0E40C62A}"
}


In [9]:
print(sail.core.wait_for_all_secure_nodes_to_be_provisioned(45000))

{
    "Failed": {

    },
    "Succeeded": {
        "{43A800D3-CF11-4BA0-99ED-A0EA0E40C62A}": ""
    },
    "InProgress": {

    },
    "AllDone": true
}


In [10]:
############################################################################
#
# Set the parameters for the job and set a variable for its output
#
############################################################################

set_ds_response = sail.core.set_parameter(job_id, "4DC7BEC15B384FBD9BD721B7BF36A984", my_table_guid)
#set_ds_response = sail.core.set_parameter(second_job_id, "4C8041B4513A44FAA5409662E014519C", provision_ds)
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")
print("First job IP {}".format(sail.core.get_ip_for_job(job_id)))
#print("Second job IP {}".format(sail.core.get_ip_for_job(second_job_id)))

First job IP 20.25.63.26


In [11]:
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")

#pull_data_result = sail.core.pull_data(data_id)
#print(pull_data_result)

In [13]:
############################################################################
#
# Produce data frame from CSV
#
############################################################################

import base64
import pickle

message_response = sail.core.wait_for_data(10000)
data_frame = None;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Dataframe has been processed from CSV")
            data_frame = un_serialized_value
            
    message_response = sail.core.wait_for_data(2000)


Job 4AF0483A580C40159995BF48AC51820D is done


In [14]:
############################################################################
#
# Get the sum of a column
#
############################################################################
sum_column_job_id = sail.core.run_job(dg_sum_column_guid)

column_name = "BMI"
# Setup a job to injest a dataframe, and sum a column's name
pushed_column_name = sail.core.push_user_data(column_name)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4C3F2D549B2042E291C47A074A0194A6", data_id)
set_ds_response = sail.core.set_parameter(sum_column_job_id, "4C275957B5C34C20BD0E02E03330167D", pushed_column_name)
print("Column job IP {}".format(sail.core.get_ip_for_job(sum_column_job_id)))
column_result_id = "{}.{}".format(sum_column_job_id, "5036707EAABB46C584FC5DBA75D5718E")

pull_data_result = sail.core.pull_data(column_result_id)

Column job IP None


In [15]:
# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(20000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("The sum of column {} is {}".format(column_name, un_serialized_value))
            
    message_response = sail.core.wait_for_data(5000)

The sum of column BMI is 7417


In [ ]:
sail.core.deprovision_digital_contract(provision_dc)

In [ ]:
sail.core.exit_current_session()

In [ ]:
safe_functions = sail.core.get_safe_functions()
if ( safe_functions is not None):
    json_safe_functions = json.loads(safe_functions)
    print(json.dumps(json_safe_functions,indent=4))
else:
    print("No safe functions")

In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
tables = sail.core.get_tables()
if ( tables is not None):
    json_tables = json.loads(tables)
    print(json.dumps(json_tables, indent=4))
else:
    print("No tables")


In [ ]:
job_id = sail.core.run_job(dg_test_object_guid)
print(job_id)

In [ ]:
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
my_table_guid = "25EC141A-735D-E94F-B01F-B66726C14034"

increment_job_id = sail.core.run_job(dg_increment_safefunction_guid)
print(increment_job_id)

pull_data_id = "{}.{}".format(increment_job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

#pull_data_result = sail.core.pull_data(pull_data_id)
#print(pull_data_result)

my_int_guid = sail.core.push_user_data(545)
set_response = sail.core.set_parameter(increment_job_id, "4C21EDF165AB4EF68853FA05FB805A03", my_int_guid)
set_response = sail.core.set_parameter(increment_job_id, "4C8041B4513A44FAA5409662E014519C", my_table_guid)
print(my_int_guid)

print("First job IP {}".format(sail.core.get_ip_for_job(increment_job_id)))

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(1000)

In [ ]:
second_increment_job_id = sail.core.run_job(dg_increment_safefunction_guid)

# We want the second job's value to be the result of the first job
second_pull_data_id = "{}.{}".format(second_increment_job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

second_pull_data_result = sail.core.pull_data(second_pull_data_id)

set_response = sail.core.set_parameter(second_increment_job_id, "4C21EDF165AB4EF68853FA05FB805A03", pull_data_id)
set_response = sail.core.set_parameter(second_increment_job_id, "4C8041B4513A44FAA5409662E014519C", my_table_guid)

print("Second job IP {}".format(sail.core.get_ip_for_job(second_increment_job_id)))

# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(1000)

In [ ]:
# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(10000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Data received {} {}".format(message_json["ValueName"], un_serialized_value))
            increment_result = un_serialized_value
            
    message_response = sail.core.wait_for_data(500)

In [ ]:
pull_data_id = "{}.{}".format(job_id, "51DD83834C2A41D6BB1F49889EA0A0AC")

pull_data_result = sail.core.pull_data(pull_data_id)
print(pull_data_result)

In [4]:
digital_contracts = sail.core.get_digital_contracts()
provision_dc = ""
if ( digital_contracts is not None):
    json_digital_contracts = json.loads(digital_contracts)
    provision_dc = list(json_digital_contracts.keys())[0]
    print(provision_dc)
    print(json.dumps(json_digital_contracts, indent=4))
else:
    print("No digital contracts")

{1429879E-8EDD-48F2-976A-CB8A2F9A3712}
{
    "{1429879E-8EDD-48F2-976A-CB8A2F9A3712}": {
        "ContractStage": 3.0,
        "DOOName": "Cancer Research Hospital",
        "ResearcherOrganization": "{04969C3A-77B2-452F-9DFB-1A7FFE6CA56D}",
        "HostForVirtualMachines": "SAIL",
        "DatasetGuid": "{222A023E-892A-452E-8358-D33517372709}",
        "VersionNumber": "0x0000000100000001",
        "ExpirationTime": 1658857713.0,
        "EulaAcceptedByROAuthorizedUser": "Company grants You a revocable, non-exclusive, non-transferable, limited right to use the dataset on a single machine.",
        "ROName": "International Genetics Research Facility",
        "ProvisioningStatus": 3.0,
        "DatasetDRMMetadataSize": 0.0,
        "AzureTemplateGuid": "{00000000-0000-0000-0000-000000000000}",
        "Description": "The dataset will be used to train models for academic research purposes.",
        "Note": "...",
        "DatasetDRMMetadata": {
            "__IsEmpty__": true
       

In [ ]:
datasets = sail.core.get_datasets()
provision_ds = ""
if ( datasets is not None ):
    json_datasets = json.loads(datasets)
    provision_ds = list(json_datasets.keys())[0]
    print(provision_ds)
    print(json.dumps(json_datasets, indent=4))
    for dataset in json_datasets:
        if dataset[1]
else:
    print("No loaded datasets")